In [58]:
!pip install -U spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.3.0/pt_core_news_sm-3.3.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


2022-06-18 19:43:41.491634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-18 19:43:41.491852: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.3.0/en_core_web_sm-3.3.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-06-18 19:43:47.949465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-18 19:43:47.949735: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [59]:
import spacy
import pt_core_news_sm

!pip install deplacy stanza
import stanza
nlp=stanza.Pipeline("pt")

2022-06-18 19:43:56 INFO: Loading these models for language: pt (Portuguese):
| Processor    | Package |
--------------------------
| tokenize     | bosque  |
| mwt          | bosque  |
| pos          | bosque  |
| lemma        | bosque  |
| depparse     | bosque  |
| constituency | cintil  |

2022-06-18 19:43:56 INFO: Use device: cpu
2022-06-18 19:43:56 INFO: Loading: tokenize
2022-06-18 19:43:56 INFO: Loading: mwt
2022-06-18 19:43:56 INFO: Loading: pos
2022-06-18 19:43:57 INFO: Loading: lemma
2022-06-18 19:43:57 INFO: Loading: depparse
2022-06-18 19:43:57 INFO: Loading: constituency
2022-06-18 19:43:57 INFO: Done loading processors!


In [60]:
#dataset = pd.read_csv('Tweets_Mg.csv',encoding='utf-8')
#dataset.count()

In [61]:
import random
#tweets = dataset["Text"].values

In [62]:
TRAIN_DATA = [
    ("Almira é uma ótima pessoa, gosto muito dela", {"entities": [(0, 6, "PER")]}),
    ("João foi para Bahia nas férias", {"entities": [(14, 19, "LOC"), (0, 4, "PER")]}),
    ("Carlos foi visitar João na casa de praia", {"entities": [(0, 6, "PER"), (19, 23, "PER")]}),
    ("No meio do ano irei para São Paulo fazer mais um curso", {"entities": [(25, 34, "LOC")]}),
    ("O sonho dela era ir para Austrália visitar seu irmão", {"entities": [(25, 34, "LOC")]}),
    ("Em 15/07/1988 nasceu essa linda criança", {"entities": [(3, 13, "DATE"), (18, 24, "LOC")]}),
    ("Data de prisão: 10/01/2018", {"entities": [(16, 26, "DATE")]}),
    ("No dia 01/02/2016 foi decretada a sentença", {"entities": [(7, 17, "DATE")]}),
    ("A data da festa foi 07/05/2018", {"entities": [(20, 30, "DATE")]}),
    ("Dia 07/06/2020 choveu pela manhã", {"entities": [(4, 14, "DATE")]}),
    ("Michael Jackson nasceu dia 29/08/1958", {"entities": [(0, 15, "PER"), ( 27, 37, "DATE")]}),
    ("A viagem foi marcada para o dia 21/03/2021", {"entities": [(32, 42, "DATE")]}),
    ("O casamento, que era dia 12/06/2020, foi adiado para o dia 20/07/2020", {"entities": [(25, 35, "DATE"), (59, 68, "DATE")]}),
    ("Dia 21/04/2020 foi feriado", {"entities": [(4, 14, "DATE")]}),
    ("A entrega foi realizada na manhã do dia 18/03/2020 ", {"entities": [(40, 50, "DATE")]}),
    ("Na noite do dia 17/05/2019 aconteceu um eclipse", {"entities": [(16, 26, "DATE")]}),
    ("Fortaleza é a capital do Ceará", {"entities": [(0, 9, "LOC"), (25, 30, "LOC")]})
]

In [63]:
def train_spacy(data, n_iter):
  TRAIN_DATA = data
  """Carrega o modelo, configura o pipeline e treina o reconhecedor da entidade."""
  nlp = pt_core_news_sm.load()  # Carrega um modelo do spaCy existente
  print("Modelo carregado")

  # Criando os componentes internos do pipeline e adicionando-os ao pipeline
  # nlp.create_pipe funciona para built-ins registrados com spaCy
  if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
  # Caso contrário, obtenha-o para que possa adicionar as labels
  else:
    ner = nlp.get_pipe("ner")

  # Adicionando as labels
  for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])

  # Obtendo os nomes de outros pipes para desativá-los durante o treinamento
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):  # Treina somente NER
    spacy.util.fix_random_seed()
    optimizer = nlp.begin_training()
    
    for itn in range(n_iter):
      random.seed(10)
      random.shuffle(TRAIN_DATA)
      losses = {}
      
      for texts, annotations in TRAIN_DATA:
        nlp.update(
            [texts],  # batch de textos
            [annotations],  # batch de anotações
            drop=0.2,  # dropout - dificulta a memorização de dados
            sgd=optimizer,
            losses=losses,
        )
      print("Losses", losses)

   # test the trained model
    for text, _ in TRAIN_DATA:
      doc = nlp(text)
      print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
      print("Tokens", [(t.text, t.ent_type_) for t in doc])
      print("\n")
    return nlp

In [64]:
nlp_ = train_spacy(data=TRAIN_DATA, n_iter=100)

Modelo carregado


ValueError: [E955] Can't find table(s) lexeme_norm for language 'pt' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.